In [362]:
'''
File Name: Digitize Committee Files.ipynb
Purpose: digitize all of committee files (type .docx, .csv/.xlsx, .pdf ONLY --> for .xls convert to .csv or .xlsx)
'''



'\nFile Name: Digitize Committee Files.ipynb\nPurpose: digitize all of committee files (type .docx, .csv/.xlsx, .pdf ONLY --> for .xls convert to .csv or .xlsx)\n'

In [54]:
import pandas as pd
import os
import glob
from docx import Document
import xlrd
import csv
import shutil

In [355]:
'''
Function Name: extract_word_data
Purpose: extract data from table in word document
PARAM: doc (opened word document using docx library --> doc = Document(file_path)
RETURN: list with all information from table
'''

'\nFunction Name: extract_word_data\nPurpose: extract data from table in word document\nPARAM: doc (opened word document using docx library --> doc = Document(file_path)\nRETURN: list with all information from table\n'

In [44]:
def extract_word_data(doc):
    # Extracts data from all tables in a Word document.
    data = []
    table = doc.tables[0] # only one table in document

    # iterate over each row in the table
    for row in table.rows: # start from second row -- ignore headers -- add headers later // remove the blank row at the bottom of each table
        row_data = []
        # iterate over each row in the table
        for cell in row.cells:
            row_data.append(cell.text)
        # append to main data list  
        data.append(row_data)
    
    return data

In [ ]:
#New
def extract_word_data_with_exception(doc):
    # Extracts data from all tables in a Word document.
    data = []
    
    # Check if the document has any tables
    if not doc.tables:
        print("No tables found in the document.")
        return data
    
    try:
        table = doc.tables[0]  # only one table in document
        
        # Iterate over each row in the table
        for row in table.rows:  # start from second row -- ignore headers -- add headers later
            row_data = []
            
            # Iterate over each cell in the row
            for cell in row.cells:
                row_data.append(cell.text)
            
            # Append row data to the main data list
            data.append(row_data)
            
    except Exception as e:
        print(f"An error occurred while processing the table: {e}")
    
    return data

In [357]:
'''
Function Name: digitize_files
Purpose: turns committee rosters (type .docx, .csv/.xlsx, .pdf) into excel documents and placed in respective subfolders (each committee) within a given folder
PARAM: 
    folder_path --> path to folder with all committee rosters
                    rosters should be seperated by committee
                    ex:
                        Committee Rosters (parent folder)
                            |--- Annual Meeting - Clincial Trials Committee
                            |        |--- 2010 - 2011 Clinical Trials Comm Roster
                            |        |--- 2011 - 2012 Clinical Trials Comm Roster
                            |--- Annual Meeting - Education Committee
                            |        |--- 2013 - 2014 Education Committee Roster
                            |        |--- 2017 - 2018 AM Education Comm Roster

    output_folder --> path to folder to put new subfolders (each committee) and the converted rosters
                      this folder should be empty -- no need to make subfolders
RETURN: none
'''

'\nFunction Name: digitize_files\nPurpose: turns committee rosters (type .docx, .csv/.xlsx, .pdf) into excel documents and placed in respective subfolders (each committee) within a given folder\nPARAM: \n    folder_path --> path to folder with all committee rosters\n                    rosters should be seperated by committee\n                    ex:\n                        Committee Rosters (parent folder)\n                            |--- Annual Meeting - Clincial Trials Committee\n                            |        |--- 2010 - 2011 Clinical Trials Comm Roster\n                            |        |--- 2011 - 2012 Clinical Trials Comm Roster\n                            |--- Annual Meeting - Education Committee\n                            |        |--- 2013 - 2014 Education Committee Roster\n                            |        |--- 2017 - 2018 AM Education Comm Roster\n\n    output_folder --> path to folder to put new subfolders (each committee) and the converted rosters\n      

In [55]:
def digitize_files(folder_path, output_folder):
    # os.walk(folder_path) iterates through a parent folder (all committee rosters) and all subfolders (each committee)
    for root, dirs, files in os.walk(folder_path): 
        # root --> parent folder
        # dirs --> child folders
        # files --> word documents
        
        file_name = "" # update to file name with .xlsx extension
        committee_name = ""
        
        for file in files:

            # fill with data from files
            file_data = []

            # word documents
            if (file.endswith('.docx')): 
                file_name = file.replace(".docx", ".xlsx") # blah.docx --> blah.xlsx
                
                file_path = os.path.join(root, file) # concatenate the parent and the subfolder
                print(f'Processing {file_path}') # check it is getting all word documents

                # extract the subfolder name (committee name)
                committee_name = os.path.relpath(root, folder_path)
                
                doc = Document(file_path) # open word document 
                table_data = extract_word_data_with_exception(doc) # extract table data ###TAKE OUT THE WITH_EXCEPTION IF NEEDED
                
                file_data.extend(table_data) # add individual rows to all_data -- append would create a 2d array

                
            # excel documents
            # .xls extension --> excel workbook before 2007 --> convert to csv or xlsx prior to running script
            elif (file.endswith('.csv') or file.endswith('.xlsx') or file.endswith('.xls')):
                
                file_path = os.path.join(root, file) # concatenate the parent and the subfolder
                print(f'Processing {file_path}') # check it is getting all documents

                # extract the subfolder name (committee name)
                committee_name = os.path.relpath(root, folder_path)

                if (file.endswith('.csv')):
                    # update file name --> replace extension
                    file_name = file.replace(".csv", ".xlsx")
                    
                    # read information in excel file with .csv extension
                    df = pd.read_csv(file_path, header = None) # header = None so the headers will be included in the list
                     # add information to file_data
                    file_data.extend(df.values.tolist())

                elif (file.endswith('.xlsx')):
                    file_name = file
                    df = pd.read_excel(file_path)
                    file_data.extend(df.values)

                elif (file.endswith('.xls')):
                    file_name = file
                    df = pd.read_excel(file_path)
                    file_data.extend(df.values)
                
                
            # pdf documents
            # TO DO
            elif file.endswith('.pdf'):
                pdf_source_path = os.path.join(root, file)
                pdf_target_path = os.path.join(committee_exceptions_folder, file)
                
                # Ensure the PDF folder exists
                os.makedirs(committee_exceptions_folder, exist_ok=True)
                
                # Move the PDF file to the designated folder
                shutil.move(pdf_source_path, pdf_target_path)
                print(f'Moved {file} to {committee_exceptions_folder}')
                continue

            
            # Convert the data to a DataFrame and save it to Excel
            df = pd.DataFrame(file_data)
            
            # create subfolder with committee name
            os.makedirs(os.path.join(output_folder, committee_name), exist_ok = True)

            # concatenate the parent folder with new subfolder and excel name
            excel_path = os.path.join(output_folder, committee_name, file_name)
        
            # output to excel
            df.to_excel(excel_path, index = False, header = False)
            
            

TESTING

In [57]:
###Testing Below

folder_path_test = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - Clinical Trials Committee"

output_path_test = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Committee Files Testing"


#Any word docs or pdfs that can't be scanned will be placed here for manual review
committee_exceptions_folder = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Committee Exceptions"

In [56]:
digitize_files(folder_path_test,output_path_test)

Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - Clinical Trials Committee\AM Clinical Trials Committee Description and History.docx
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - Clinical Trials Committee\2010-2011 Clinical Trials Committee\2010-2011 Clinical Trials Comm Roster.docx
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - Clinical Trials Committee\2011-2012 Clinical Trials Committee\2011-2012 AM Clinical Trials Comm - 2 Column Format.docx
No tables found in the document.
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - Clinical Trials Committee\2011-2012 Clinical Trials Committee\2011-2012 AM Clinical Trials Comm Roster.docx
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Documents\Annual Meeting - C

Production

In [22]:
# specify parent path
folder_path = r"C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents"

###Was: "C:\Users\keyra.desouza\OneDrive - American Association for Cancer Res\Committee Rosters"

# output will be a collection of committee folders with excel sheets in them for each year
output_folder = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\Committees Output Files"

###Was: "C:\Users\keyra.desouza\OneDrive - American Association for Cancer Res\Digitized Committee Rosters"


In [23]:
# process all documents 
digitize_files(folder_path, output_folder)

print()
print(f'Results in {output_folder}')

ValueError: No engine for filetype: ''

In [34]:
###This script will convert all .xls files to .csv files

for root, dirs, files in os.walk(folder_path): 
        # root --> parent folder
        # dirs --> child folders
        # files --> word documents
        
        file_name = "" # update to file name with .xlsx extension
        committee_name = ""
        
        for file in files:

            # fill with data from files
            file_data = []

            # word documents
            if (file.endswith('.xls')):
                file_path = os.path.join(root, file) 
                #file.replace('.xls', '.csv')
                print(f'XLS to convert: {file_path}') # check it is getting all word documents


XLS to convert: C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Education Committee\2014-2015 AM Education Committee\2014-2015 Education Committee Suggested Members_ Alpha Order.xls
XLS to convert: C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Education Committee\2014-2015 AM Education Committee\2015 Education Comm Suggestions as of 7.23.14.xls
XLS to convert: C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Education Committee\2014-2015 AM Education Committee\2015 Education Committee Member Invitation Tracking Report 9.17.14.xls
XLS to convert: C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Education Committee\2014-2015 AM Education Comm

In [39]:
def xls_to_csv(directory):
    # Walk through the directory and all subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".xls"):
                # Create full path to the .xls file
                xls_file_path = os.path.join(root, file)
                
                # Load the .xls file into a pandas DataFrame
                try:
                    df = pd.read_excel(xls_file_path)
                    
                    # Create the output .csv file path
                    csv_file_path = os.path.splitext(xls_file_path)[0] + ".csv"
                    
                    # Save the DataFrame as a .csv file
                    df.to_csv(csv_file_path, index=False)
                    
                    print(f"Converted: {xls_file_path} to {csv_file_path}")
                except Exception as e:
                    print(f"Failed to convert {xls_file_path}: {e}")

# Replace with the path to your folder
directory = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST"
xls_to_csv(directory)

Converted: C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2014-2015 Education Committee Suggested Members_ Alpha Order.xls to C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2014-2015 Education Committee Suggested Members_ Alpha Order.csv
Converted: C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Comm Suggestions as of 7.23.14.xls to C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Comm Suggestions as of 7.23.14.csv
Converted: C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Committee Member Invitation Tracking Report 9.17.14.xls to C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TE

In [40]:
digitize_files(directory, output_folder)

print()
print(f'Results in {output_folder}')

Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2014-2015 AM Education Comm Roster Recommendations.docx
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2014-2015 Education Committee Suggested Members_ Alpha Order.csv
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Comm Suggestions as of 7.23.14.csv
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Committee Member Invitation Tracking Report 9.17.14.csv
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST\2015 Education Committee Member Invitation Tracking Report 9.3.14.csv
Processing C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\

In [41]:
def xls_to_csv(directory):
    # Walk through the directory and all subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".xls"):
                # Create full path to the .xls file
                xls_file_path = os.path.join(root, file)
                
                # Load the .xls file into a pandas DataFrame
                try:
                    df = pd.read_excel(xls_file_path)
                    
                    # Create the output .csv file path
                    csv_file_path = os.path.splitext(xls_file_path)[0] + ".csv"
                    
                    # Save the DataFrame as a .csv file
                    df.to_csv(csv_file_path, index=False)
                    
                    print(f"Converted: {xls_file_path} to {csv_file_path}")
                except Exception as e:
                    print(f"Failed to convert {xls_file_path}: {e}")

# Replace with the path to your folder
directory = r"C:\Users\maxwell.bicking\OneDrive - American Association for Cancer Res\2014-2015 AM Education Committee TEST"
xls_to_csv(directory)

Failed to convert C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Program Committee\2012-2013 AM Program Committee\2013 AM Invited Speaker List - OLD.xls: [Errno 13] Permission denied: 'C:\\Users\\maxwell.bicking\\OneDrive - American Association for Cancer Res\\Committees Output Files'
Failed to convert C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Program Committee\2014-2015 AM Program Committee\AM2015 Complete Listing of Abstract Reviewers.xls: [Errno 13] Permission denied: 'C:\\Users\\maxwell.bicking\\OneDrive - American Association for Cancer Res\\Committees Output Files'
Failed to convert C:\Users\maxwell.bicking\American Association for Cancer Res\Scientific Programs and Strategic Initiatives - Documents\Annual Meeting - Program Committee\2014-2015 AM Program Committee\UPDATED FINAL 2015 PG with abstract 

In [ ]:
#file.replace('.xls', '.csv')

xlspath = 